In [3]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar bibliotecas
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer # Added import for ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Carregar o dataset
file_path = '/content/drive/MyDrive/Colab Notebooks/insurance.csv'

data = pd.read_csv(file_path)
print(data.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


In [6]:
# Definir X e y
X = data.drop('charges', axis=1)
y = data['charges']

# Identificar colunas categóricas e numéricas
cat_features = ['sex', 'smoker', 'region']
num_features = ['age', 'bmi', 'children']

# Pipeline para pré-processar dados categóricos (OneHotEncoder) e deixar numéricos como estão
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), cat_features),
        ('num', 'passthrough', num_features)
    ])

# Dividir dados treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Modelos a treinar
models = {
    'Árvore de Regressão': DecisionTreeRegressor(random_state=42),
    'SVM Regressor': SVR(kernel='linear'),
    'MLP Regressor': MLPRegressor(random_state=42, max_iter=500)
}

# Dicionário para armazenar resultados
results = {}

for nome, model in models.items():
    # Criar pipeline com pré-processamento e modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])
    # Treinar
    pipeline.fit(X_train, y_train)
    # Prever
    y_pred = pipeline.predict(X_test)
    # Avaliar
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[nome] = {'MAE': mae, 'MSE': mse, 'R2': r2}

# Mostrar resultados numa tabela
df_results = pd.DataFrame(results).T

# Formatar números para melhor visualização
df_formatado = df_results.copy()
df_formatado['MAE'] = df_formatado['MAE'].map(lambda x: f"{x:,.2f}")
df_formatado['MSE'] = df_formatado['MSE'].map(lambda x: f"{x:,.2f}")
df_formatado['R2'] = df_formatado['R2'].map(lambda x: f"{x:.4f}")

print('\nResultados dos Modelos:')
print(df_formatado)


Resultados dos Modelos:
                          MAE             MSE       R2
Árvore de Regressão  2,821.93   38,315,168.64   0.7387
SVM Regressor        6,351.01  161,142,758.67  -0.0990
MLP Regressor        9,096.82  126,782,118.79   0.1353


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


O modelo de Árvore de Regressão apresentou o melhor desempenho, com o menor erro médio absoluto (MAE) e erro quadrático médio (MSE), além de um coeficiente de determinação (R²) bastante elevado (0,74), indicando que ele explica cerca de 74% da variação dos valores do seguro. Isso mostra que o modelo capturou bem os padrões dos dados.

Já o modelo SVM teve o pior desempenho, com erros muito maiores e um R² negativo, indicando que seu desempenho é pior do que simplesmente usar a média para predizer o valor do seguro. O modelo MLP teve um desempenho intermediário, com erro relativamente alto e um R² baixo, sugerindo que ainda não conseguiu aprender bem as relações presentes nos dados.